In [1]:
import numpy as np
import pywt
import os
from PIL import Image
from scipy.fftpack import dct
from scipy.fftpack import idct
import cv2

In [2]:
image = 'image_with_watermark.png'   
watermark = 'wm_rosace.jpg' 

<img src="pictures/imagetest1.jpg" width="400" />
Watermarkded image
<img src="pictures/wmtest1.jpg">
Extracted watermark

In [3]:
def convert_image(image_name, size):
    img = Image.open(image_name)
    print(type(img))
    img = img.resize((size, size), 1)
    img = img.convert('L')
    img.save(image_name)

 
    image_array = np.array(img.getdata(), dtype=np.float32).reshape((size, size))
    print(image_array[0][0])
    print(image_array[10][10])             

    return image_array

In [4]:
def process_coefficients(imArray, model, level):
    coeffs=pywt.wavedec2(data = imArray, wavelet = model, level = level)
    # print coeffs[0].__len__()
    coeffs_H=list(coeffs) 
   
    return coeffs_H


In [5]:
def embed_mod2(coeff_image, coeff_watermark, offset=0):
    for i in xrange(coeff_watermark.__len__()):
        for j in xrange(coeff_watermark[i].__len__()):
            coeff_image[i*2+offset][j*2+offset] = coeff_watermark[i][j]

    return coeff_image

In [6]:
def embed_mod4(coeff_image, coeff_watermark):
    for i in xrange(coeff_watermark.__len__()):
        for j in xrange(coeff_watermark[i].__len__()):
            coeff_image[i*4][j*4] = coeff_watermark[i][j]

    return coeff_image

In [7]:
def embed_watermark(watermark_array, orig_image, coeff):
    watermark_array_size = watermark_array[0].__len__()
    watermark_flat = watermark_array.ravel()
    ind = 0

    for x in range (0, orig_image.__len__(), 8):
        for y in range (0, orig_image.__len__(), 8):
            if ind < watermark_flat.__len__():
                subdct = orig_image[x:x+8, y:y+8]
                subdct[5][5] = watermark_flat[ind] * coeff
                orig_image[x:x+8, y:y+8] = subdct
                ind += 1 


    return orig_image

In [8]:
def apply_dct(image_array):
    size = image_array[0].__len__()
    all_subdct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subpixels = image_array[i:i+8, j:j+8]
            subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")
            all_subdct[i:i+8, j:j+8] = subdct

    return all_subdct

In [9]:
def inverse_dct(all_subdct):
    size = all_subdct[0].__len__()
    all_subidct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")
            all_subidct[i:i+8, j:j+8] = subidct

    return all_subidct

In [10]:
def get_watermark(dct_watermarked_coeff, watermark_size, coeff):
    
    subwatermarks = []

    for x in range (0, dct_watermarked_coeff.__len__(), 8):
        for y in range (0, dct_watermarked_coeff.__len__(), 8):
            coeff_slice = dct_watermarked_coeff[x:x+8, y:y+8]
            subwatermarks.append(coeff_slice[5][5]/coeff)

    watermark = np.array(subwatermarks).reshape(watermark_size, watermark_size)

    return watermark

In [11]:
def recover_watermark(image_array, model='haar', level = 1):

    coeffs_watermarked_image = process_coefficients(image_array, model, level=level)
    dct_watermarked_coeff = apply_dct(coeffs_watermarked_image[0])
    
    watermark_array = get_watermark(dct_watermarked_coeff, 128, 0.1)

    watermark_array =  np.uint8(watermark_array)

#Save result
    img = Image.fromarray(watermark_array)
    img.save('./recovered_watermark.jpg')

In [12]:
def print_image_from_array(image_array, name):
  
    image_array_copy = image_array.clip(0, 255)
    image_array_copy = image_array_copy.astype("uint8")
    img = Image.fromarray(image_array_copy)
    img.save(name)

In [13]:
def add_gaussian_noise(image, mean, std_dev):
    row, col = image.shape
    gauss = np.random.normal(mean, std_dev, (row, col))
    gauss = gauss.reshape(row, col)
    noisy = image + gauss
    print_image_from_array(noisy, 'image_w_gaussian.png')

    return noisy

In [15]:
def w2d(img):
    model = 'haar'
    level = 1
    image_array = convert_image(image, 2048)
    watermark_array = convert_image(watermark, 128)

    coeffs_image = process_coefficients(image_array, model, level=level)
    dct_array = apply_dct(coeffs_image[0])
    dct_array = embed_watermark(watermark_array, dct_array, 0.1)
    coeffs_image[0] = inverse_dct(dct_array)
  

# reconstruction
    image_array_H=pywt.waverec2(coeffs_image, model)
    print_image_from_array(image_array_H, 'image_with_watermark.png')
    
    
# recover images
    recover_watermark(image_array = image_array_H, model=model, level = level)

In [35]:
w2d("test")

<class 'PIL.PngImagePlugin.PngImageFile'>
144.0
146.0
<class 'PIL.JpegImagePlugin.JpegImageFile'>
255.0
255.0


<img src="result/image_with_watermark.jpg" width="400" />
Watermarkded image
<img src="result/recovered_watermark.jpg">
Extracted watermark

In [16]:
def translation(image, k):
    cop = image
    for x in range(0, len(cop)):
        for y in range(0, len(cop[0])):
            cop[x][y]=image[x][(y+k)%len(cop[0])]
    print_image_from_array(cop, 'image_translated.png')
    return cop

In [14]:
def cropping(image): 
    img = cv2.imread(image)
    y=4
    x=4
    h=len(img)-4
    w=len(img)-4
    crop_image = img[x:w, y:h]
    return crop_image

In [15]:
def convert_image_2(img, size):
    img = img.resize((size, size), 1)
    img = img.convert('L')
    img.save('cropped.png')

 
    image_array = np.array(img.getdata(), dtype=np.float32).reshape((size, size))
    print(image_array[0][0])
    print(image_array[10][10])             

    return image_array

In [16]:
def random_coordinate(t, img):
    x = np.arange(0, t)
    y = np.arange(0, t)
    for i in range(0, t):
        x[i] = np.random.randint(0, len(img))
        y[i] = np.random.randint(0, len(img))
    return x, y

In [17]:
def exchange(img, t):
    ramdomised = img 
    x1, y1 = random_coordinate(t=t, img=img)
    x2, y2 = random_coordinate(t=t, img=img)
    for i in range(0, t):
        ramdomised[x1[i]][y1[i]]=img[x2[i]][y2[i]]
    return ramdomised


In [18]:
def extract(t):
    model = 'haar'
    level = 1
    image_array = convert_image(image, 2048)
    randomised=exchange(image_array, t)
    print_image_from_array(randomised, 'image_randomised_'+str(t)+'.png')
    recover_watermark(image_array = image_array, model=model, level = level)
    

In [20]:
extract(1000)

<class 'PIL.PngImagePlugin.PngImageFile'>
144.0
146.0
